#### Script to produce growth values and growth factors for EFS at required segmentation

In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
# POPULATION DATA:
population_file_path = r'Y:\NorMITs Demand\inputs\scenarios\Population\Population_growth_CSV'

SC01_JAM_Population = pd.read_csv(population_file_path + '\SC01_JAM_population_growth_MSOA.csv')
SC02_PP_Population = pd.read_csv(population_file_path + '\SC02_PP_population_growth_MSOA.csv')
SC03_DD_Population = pd.read_csv(population_file_path + '\SC03_DD_population_growth_MSOA.csv')
SC04_UZC_Population = pd.read_csv(population_file_path + '\SC04_UZC_population_growth_MSOA.csv')

In [ ]:
# EMPLOYMENT DATA:
employment_file_path = r'Y:\NorMITs Demand\inputs\scenarios\Employment\Job_growth_CSV'

SC01_JAM_Job = pd.read_csv(employment_file_path + '\SC01_JAM_job_growth_MSOA.csv')
SC02_PP_Job = pd.read_csv(employment_file_path + '\SC02_PP_job_growth_MSOA.csv')
SC03_DD_Job = pd.read_csv(employment_file_path + '\SC03_DD_job_growth_MSOA.csv')
SC04_UZC_Job = pd.read_csv(employment_file_path + '\SC04_UZC_job_growth_MSOA.csv')

In [ ]:
# Percentage of people per unique row for growth values calculation:
segment_file_path = r'Y:\NorMITs Demand\inputs\scenarios\EFS inputs'

# msoa,sec,soc (for population, from land use)
sec_soc_segment = pd.read_csv(segment_file_path + '\merged_totals.csv')

# msoa,soc (for employment)
soc_segment = pd.read_excel(segment_file_path + '\SOC by skill (uplifted).xlsx', sheet_name = 'pivot')

#### Growth values for EFS:  apply percentage splits producing output per MSOA, SEC, SOC (population) and MSOA,SOC growth values (employment) 

In [ ]:
def calculate_segmented_growth(inputdata, segmentation):
    
    inputdata = inputdata.copy()
    segmentation = segmentation.copy()
    
    # merge % of people in each unique row to each scenario:
    inputdata = pd.merge(segmentation, inputdata,
                          left_on = ['msoa_zone_id'],
                          right_on = ['msoa_zone_id'])
    
    
    for year in range(int(2018), int(2051)):
        inputdata[str(year)] = inputdata['% of MSOA_people_sum'] * inputdata[str(year)] / 100

    # drop unnecessary cols        
    cols_to_drop = ['MSOA people sum_x','North', 'Unique row people sum', 'Unique row people sum_y', 
                    'MSOA workers sum'] 
    for column_name in cols_to_drop:
        if column_name in inputdata.columns:
            inputdata.drop([column_name] , axis='columns', inplace=True)
    
    # tidy col names
    inputdata.rename(columns = {'MSOA people sum_y':'MSOA_people_sum'}, inplace = True) 
    inputdata.rename(columns = {'soc_cat':'soc'}, inplace = True) 
    inputdata.rename(columns = {'ns_sec':'ns'}, inplace = True) 

    return inputdata
    

In [ ]:
SC01_JAM_pop_segments_growth = calculate_segmented_growth(SC01_JAM_Population, segmentation = sec_soc_segment)
SC02_PP_pop_segments_growth = calculate_segmented_growth(SC02_PP_Population, segmentation = sec_soc_segment)
SC03_DD_pop_segments_growth = calculate_segmented_growth(SC03_DD_Population, segmentation = sec_soc_segment)
SC04_UZC_pop_segments_growth = calculate_segmented_growth(SC04_UZC_Population, segmentation = sec_soc_segment)

In [ ]:
SC01_JAM_job_segments_growth = calculate_segmented_growth(SC01_JAM_Job, segmentation = soc_segment)
SC02_PP_job_segments_growth = calculate_segmented_growth(SC02_PP_Job, segmentation = soc_segment)
SC03_DD_job_segments_growth = calculate_segmented_growth(SC03_DD_Job, segmentation = soc_segment)
SC04_UZC_job_segments_growth = calculate_segmented_growth(SC04_UZC_Job, segmentation = soc_segment)

In [ ]:
# segmented outputs for population
SC01_JAM_pop_segments_growth.to_csv('SC01_JAM_pop_segmented_EFS.csv', index=False)
SC02_PP_pop_segments_growth.to_csv('SC02_PP_pop_segmented_EFS.csv', index=False)
SC03_DD_pop_segments_growth.to_csv('SC03_DD_pop_segmented_EFS.csv', index=False)
SC04_UZC_pop_segments_growth.to_csv('SC04_UZC_pop_segmented_EFS.csv', index=False)

In [ ]:
# segmented outputs for jobs
SC01_JAM_job_segments_growth.to_csv('SC01_JAM_job_segmented_EFS_soc.csv', index=False)
SC02_PP_job_segments_growth.to_csv('SC02_PP_job_segmented_EFS_soc.csv', index=False)
SC03_DD_job_segments_growth.to_csv('SC03_DD_job_segmented_EFS_soc.csv', index=False)
SC04_UZC_job_segments_growth.to_csv('SC04_UZC_job_segmented_EFS_soc.csv', index=False)

#### Calculate growth factors: future year/base year(2018) 

In [ ]:
def calculate_growth_factors(inputdata):
    
    inputdata = inputdata.copy()
    
    outputdata = inputdata.copy()
    outputdata['2018'] = 1
    
    for year in range(int(2019), int(2051)):
        outputdata[str(year)] = inputdata[str(year)] / inputdata[str(2018)]
        
        # if 0%, fill row with growth factors of 1
        outputdata.loc[outputdata['% of MSOA_people_sum'] == 0, str(year)] = 1

    return outputdata 

In [ ]:
GF_SC01_JAM_pop =  calculate_growth_factors(SC01_JAM_pop_segments_growth) 
GF_SC02_PP_pop =  calculate_growth_factors(SC02_PP_pop_segments_growth) 
GF_SC03_DD_pop =  calculate_growth_factors(SC03_DD_pop_segments_growth) 
GF_SC04_UZC_pop =  calculate_growth_factors(SC04_UZC_pop_segments_growth) 

In [ ]:
GF_SC01_JAM_jobs =  calculate_growth_factors(SC01_JAM_job_segments_growth) 
GF_SC02_PP_jobs =  calculate_growth_factors(SC02_PP_job_segments_growth) 
GF_SC03_DD_jobs =  calculate_growth_factors(SC03_DD_job_segments_growth) 
GF_SC04_UZC_jobs =  calculate_growth_factors(SC04_UZC_job_segments_growth) 

In [ ]:
GF_SC01_JAM_pop.to_csv('SC01_JAM_pop_growthfactors_EFS.csv', index=False)
GF_SC02_PP_pop.to_csv('SC02_PP_pop_growthfactors_EFS.csv', index=False)
GF_SC03_DD_pop.to_csv('SC03_DD_pop_growthfactors_EFS.csv', index=False)
GF_SC04_UZC_pop.to_csv('SC04_UZC_pop_growthfactors_EFS.csv', index=False)

In [ ]:
GF_SC01_JAM_jobs.to_csv('SC01_JAM_job_growthfactors_EFS_soc.csv', index=False)
GF_SC02_PP_jobs.to_csv('SC02_PP_job_growthfactors_EFS_soc.csv', index=False)
GF_SC03_DD_jobs.to_csv('SC03_DD_job_growthfactors_EFS_soc.csv', index=False)
GF_SC04_UZC_jobs.to_csv('SC04_UZC_job_growthfactors_EFS_soc.csv', index=False)